# 🦌 ELK Transformer 🤖

A transformer object that will convert some input source into valid Elk Json. This
example using a transformer around a networkx graph.

In [ ]:
import json
import pathlib

import ipywidgets
import networkx
import traitlets
from IPython.display import display

import ipyelk
import ipyelk.nx
import ipyelk.tools

## Flat structure

A `networkx.MultiDigraph` can be used to create a flat graph.

> _TODO: There should be an option to specify if ports should be created or only connect
> edges between the nodes_

In [ ]:
def a_flat_elk_json_example(graph: networkx.MultiDiGraph = None):
    graph = graph or networkx.readwrite.json_graph.node_link_graph(
        json.loads(pathlib.Path("flat_graph.json").read_text(encoding="utf-8"))
    )

    elk = ipyelk.ElkDiagram(layout=dict(min_height="200px"))
    xelk = ipyelk.nx.XELK(source=(graph, None))
    xelk.connect(elk)
    return elk, xelk

In [ ]:
if __name__ == "__main__":
    flat, xflat = a_flat_elk_json_example()
    display(flat)

## Hierarchical Diagram with Ports

In [ ]:
def a_hierarchical_elk_example(
    tree: networkx.MultiDiGraph = None, ports: networkx.MultiDiGraph = None
):
    tree = tree or networkx.readwrite.json_graph.node_link_graph(
        json.loads(pathlib.Path("hier_tree.json").read_text(encoding="utf-8"))
    )
    ports = ports or networkx.readwrite.json_graph.node_link_graph(
        json.loads(pathlib.Path("hier_ports.json").read_text(encoding="utf-8"))
    )

    elk = ipyelk.ElkDiagram()
    xelk = ipyelk.nx.XELK(source=(ports, tree))
    xelk.connect(elk)

    return elk, xelk

In [ ]:
def a_collapsible_elk_example(elk_xelk=None):
    elk, xelk = elk_xelk or a_hierarchical_elk_example()
    ports, tree = xelk.source

    toggle = ipywidgets.Button(description="Toggle Collapsed")

    @toggle.on_click
    def toggle_node(widget):
        for element_id in elk.selected:
            if element_id in tree:
                for child in tree.neighbors(element_id):
                    state = tree.nodes[child].get("hidden", False)
                    tree.nodes[child]["hidden"] = not state
                xelk.refresh()

    box = ipywidgets.VBox(
        [
            ipywidgets.HBox(
                [ipywidgets.HTML("<h2>👇 click a group node then click 👉</h2>"), toggle]
            ),
            elk,
        ]
    )
    return box, elk, xelk

In [ ]:
if __name__ == "__main__":
    hier_box, hier_elk, hier_xelk = a_collapsible_elk_example()
    display(hier_box)

## 🦌 Learn More 📖

See the [other examples](./_index.ipynb).